In [0]:
from datetime import datetime
print("Current Date/Time:",datetime.now())

Current Date/Time: 2020-05-30 21:16:21.119475


In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
workingdir = '/content/gdrive/My Drive/Colab Notebooks/EVA4/Session9'
sys.path.append(workingdir)

In [0]:
%matplotlib inline
import torch
from utils import datapreparation
from utils import display
from utils import training
from utils import evaluate
from model import QuizDNN

In [5]:
classes, trainloader, testloader = datapreparation.load()

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
CUDA Available? True


# PASTING THE CODE FROM QuizDNN.py

In [0]:
# PASTING THE CODE FROM QuizDNN.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Input Block
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64))

        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64))

        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64))

        self.pool1 = nn.Sequential(nn.MaxPool2d(2, 2))

        self.conv4 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64))

        self.conv5 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64))

        self.conv6 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64))

        self.pool2 = nn.Sequential(nn.MaxPool2d(2, 2))

        self.conv7 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64))

        self.conv8 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64))

        self.conv9 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64))

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=8))

        self.conv10 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=10, kernel_size=(1, 1), padding=0, bias=False))

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x1 + x2)

        x4 = self.pool1(x1 + x2 + x3)

        x5 = self.conv4(x4)
        x6 = self.conv5(x4 + x5)
        x7 = self.conv6(x4 + x5 + x6)

        x8 = self.pool2(x5 + x6 + x7)

        x9 = self.conv7(x8)
        x10 = self.conv8(x8 + x9)
        x11 = self.conv9(x8 + x9 + x10)

        x12 = self.gap(x11)
        x13 = self.conv10(x12)

        x13 = x13.view(-1, 10)

        return F.log_softmax(x13, dim=-1)


In [7]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
              ReLU-2           [-1, 64, 32, 32]               0
       BatchNorm2d-3           [-1, 64, 32, 32]             128
            Conv2d-4           [-1, 64, 32, 32]          36,864
              ReLU-5           [-1, 64, 32, 32]               0
       BatchNorm2d-6           [-1, 64, 32, 32]             128
            Conv2d-7           [-1, 64, 32, 32]          36,864
              ReLU-8           [-1, 64, 32, 32]               0
       BatchNorm2d-9           [-1, 64, 32, 32]             128
        MaxPool2d-10           [-1, 64, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]          36,864
             ReLU-12           [-1, 64, 16, 16]               0
      BatchNorm2d-13           [-1, 64, 16, 16]             128
           Conv2d-14           [-1

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

In [10]:
EPOCHS = 10
for epoch in range(EPOCHS):
    print("EPOCH:", epoch+1)
    training.train_model(model, device, trainloader, optimizer, criterion, epoch)
    training.test_model(model, device, criterion, testloader)

  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 1


Loss=1.143080234527588 Batch_id=390 Accuracy=46.38: 100%|██████████| 391/391 [00:20<00:00, 18.94it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.0097, Accuracy: 5548/10000 (55.48%)

EPOCH: 2


Loss=1.1586912870407104 Batch_id=390 Accuracy=60.79: 100%|██████████| 391/391 [00:20<00:00, 18.83it/s]
  0%|          | 0/391 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0043, Accuracy: 8159/10000 (81.59%)

EPOCH: 9


Loss=0.7058678865432739 Batch_id=390 Accuracy=80.50: 100%|██████████| 391/391 [00:20<00:00, 19.54it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.0042, Accuracy: 8222/10000 (82.22%)

EPOCH: 10


Loss=0.4551335275173187 Batch_id=390 Accuracy=81.45: 100%|██████████| 391/391 [00:19<00:00, 19.74it/s]



Test set: Average loss: 0.0040, Accuracy: 8273/10000 (82.73%)

